## Configuración Inicial

Define tus rutas de usuario y proyecto aquí. Estas variables se utilizarán en todo el notebook.

In [ ]:
# ========================================
# CONFIGURACIÓN DE RUTAS
# ========================================
# Modifica estos valores según tu configuración

USER_PATH = 'users/tu_usuario'              # Ruta de tu usuario personal en GEE
PROJECT_PATH = 'projects/tu-proyecto/assets'  # Ruta de tu proyecto en GEE

# Ejemplos de uso:
# USER_PATH = 'users/cnav'
# PROJECT_PATH = 'projects/ee-ugr-default-project/assets'

## 1. Autenticación e Inicialización

### 1.1 Autenticación básica

In [ ]:
import ee

# Autenticación forzada (abre ventana de navegador)
ee.Authenticate(force=True)
ee.Initialize()

print("Autenticación exitosa")

### 1.2 Autenticación con permisos completos

Si necesitas permisos adicionales para operaciones de copia/eliminación, usa esta opción con todos los scopes.

In [ ]:
import ee

ee.Authenticate(
    force=True,
    scopes=[
        'https://www.googleapis.com/auth/earthengine',
        'https://www.googleapis.com/auth/devstorage.full_control',
        'https://www.googleapis.com/auth/cloud-platform'
    ]
)
ee.Initialize()

print("Autenticación con permisos completos exitosa")

## 2. Operaciones de Consulta

### 2.1 Listar assets en una ubicación

In [ ]:
import ee
ee.Initialize()

# Listar assets en usuario personal
assets_user = ee.data.listAssets({'parent': USER_PATH})
print(f"Assets en {USER_PATH}:")
for asset in assets_user.get('assets', []):
    print(f"  - {asset['id']} ({asset['type']})")

# Listar assets en proyecto
assets_project = ee.data.listAssets({'parent': PROJECT_PATH})
print(f"\nAssets en {PROJECT_PATH}:")
for asset in assets_project.get('assets', []):
    print(f"  - {asset['id']} ({asset['type']})")

### 2.2 Verificar información de un asset

In [ ]:
import ee
ee.Initialize()

# Verificar información de un asset específico
asset_id = f'{USER_PATH}/nombre_del_asset'

try:
    info = ee.data.getAsset(asset_id)
    print(f"Tipo: {info['type']}")
    print(f"ID: {info['id']}")
    print(f"Nombre: {info['name']}")
except Exception as e:
    print(f"Error al obtener información: {e}")

### 2.3 Verificar permisos de un asset

In [ ]:
import ee
ee.Initialize()

# Verificar ACL (Access Control List) de un asset
asset_id = f'{USER_PATH}/nombre_del_asset'

try:
    acl = ee.data.getAssetAcl(asset_id)
    print("Permisos del asset:")
    print(acl)
except Exception as e:
    print(f"Error al obtener permisos: {e}")

## 3. Copiar Assets - Operaciones Individuales

### 3.1 Copiar una imagen individual

In [ ]:
import ee
ee.Initialize()

# Copiar una imagen de un lugar a otro
src_image = f'{USER_PATH}/nombre_imagen'
dst_image = f'{PROJECT_PATH}/nombre_imagen_destino'

print(f"Copiando imagen:\n  {src_image}\n  → {dst_image}")

try:
    ee.data.copyAsset(src_image, dst_image)
    print("Copia completada exitosamente")
except Exception as e:
    print(f"Error al copiar: {e}")

### 3.2 Copiar una tabla individual

In [ ]:
import ee
ee.Initialize()

# Copiar una tabla (FeatureCollection)
src_table = f'{USER_PATH}/nombre_tabla'
dst_table = f'{PROJECT_PATH}/nombre_tabla_destino'

print(f"Copiando tabla:\n  {src_table}\n  → {dst_table}")

try:
    ee.data.copyAsset(src_table, dst_table)
    print("Copia completada exitosamente")
except Exception as e:
    print(f"Error al copiar: {e}")

### 3.3 Copiar múltiples imágenes de una carpeta

In [ ]:
import ee
ee.Initialize()

# Copiar todas las imágenes de una carpeta origen a una carpeta destino
src_folder = f'{USER_PATH}/carpeta_origen'
dst_folder = f'{PROJECT_PATH}/carpeta_destino'

# Listar assets en la carpeta origen
assets = ee.data.listAssets({'parent': src_folder})['assets']

print(f"Copiando {len(assets)} assets de {src_folder} a {dst_folder}\n")

for asset in assets:
    src = asset['id']
    asset_name = src.split('/')[-1]
    dst = f'{dst_folder}/{asset_name}'
    
    print(f"Copiando: {asset_name}")
    try:
        ee.data.copyAsset(src, dst)
        print(f"  → Éxito")
    except Exception as e:
        print(f"  → Error: {e}")

print("\nProceso completado")

## 4. Copiar ImageCollections

### 4.1 Copiar ImageCollection completa (mismo tipo de ubicación)

Para copiar entre usuarios o entre proyectos, primero se crea la colección destino y luego se copian las imágenes.

In [ ]:
import ee
ee.Initialize()

# Definir origen y destino
src_collection = f'{USER_PATH}/coleccion_origen'
dst_collection = f'{USER_PATH}/coleccion_destino'

# 1. Crear la ImageCollection destino
print(f"Creando ImageCollection: {dst_collection}")
try:
    ee.data.createAsset({'type': 'ImageCollection'}, dst_collection)
    print("  → Colección creada")
except Exception as e:
    print(f"  → Error (puede que ya exista): {e}")

# 2. Copiar todas las imágenes
print(f"\nCopiando imágenes de {src_collection}...")
imgs = ee.ImageCollection(src_collection).toList(10000)
n = imgs.size().getInfo()
print(f"Total de imágenes a copiar: {n}\n")

for i in range(n):
    img = ee.Image(imgs.get(i))
    src = img.getInfo()['id']
    img_name = src.split('/')[-1]
    dst = f'{dst_collection}/{img_name}'
    
    print(f"  [{i+1}/{n}] Copiando: {img_name}")
    try:
        ee.data.copyAsset(src, dst)
        print(f"    → Éxito")
    except Exception as e:
        print(f"    → Error: {e}")

print("\nCopia de colección completada")

### 4.2 Copiar ImageCollection a carpeta (sin crear colección)

Útil para copiar imágenes de una colección a una carpeta de proyecto.

In [ ]:
import ee
ee.Initialize()

# Copiar imágenes de una ImageCollection a una carpeta
src_collection = f'{USER_PATH}/coleccion_origen'
dst_folder = f'{PROJECT_PATH}/carpeta_destino'

# Obtener lista de imágenes
image_list = ee.ImageCollection(src_collection).toList(10000)
count = image_list.size().getInfo()
print(f"Total de imágenes: {count}\n")

for i in range(count):
    img = ee.Image(image_list.get(i))
    info = img.getInfo()
    
    src_id = info['id']
    img_name = src_id.split('/')[-1]
    dst_id = f"{dst_folder}/{img_name}"

    print(f"[{i+1}/{count}] Copiando: {img_name}")
    try:
        ee.data.copyAsset(src_id, dst_id)
        print(f"  → Éxito")
    except Exception as e:
        print(f"  → Error: {e}")

print("\nCopia completada")

### 4.3 Copiar ImageCollection a proyecto (creando nueva colección)

Crea una nueva ImageCollection en el proyecto y copia todas las imágenes.

In [ ]:
import ee
ee.Initialize()

# Copiar ImageCollection completa a proyecto
src_collection = f'{USER_PATH}/coleccion_origen'
dst_collection = f'{PROJECT_PATH}/coleccion_destino'

# 1. Crear la ImageCollection en el proyecto
print(f"Creando ImageCollection: {dst_collection}")
try:
    ee.data.createAsset({'type': 'ImageCollection'}, dst_collection)
    print("  → Colección creada exitosamente")
except Exception as e:
    print(f"  → Advertencia: {e}")

# 2. Copiar todas las imágenes
print(f"\nCopiando imágenes...")
image_list = ee.ImageCollection(src_collection).toList(10000)
count = image_list.size().getInfo()
print(f"Total: {count} imágenes\n")

for i in range(count):
    img = ee.Image(image_list.get(i))
    info = img.getInfo()
    
    src_id = info['id']
    img_name = src_id.split('/')[-1]
    dst_id = f"{dst_collection}/{img_name}"

    print(f"[{i+1}/{count}] {img_name}")
    try:
        ee.data.copyAsset(src_id, dst_id)
        print(f"  → Copiado")
    except Exception as e:
        print(f"  → Error: {e}")

print("\nProceso completado")

## 5. Copiar Estructuras Completas (Recursivo)

### 5.1 Función de copia recursiva

Esta función copia carpetas completas con todas sus subcarpetas, ImageCollections, imágenes y tablas.

In [ ]:
import ee
ee.Initialize()

def exists(asset_id):
    """Verifica si un asset existe."""
    try:
        ee.data.getAsset(asset_id)
        return True
    except:
        return False

def join_path(parent, child_name):
    """Construye una ruta válida de asset en EE."""
    return parent.rstrip('/') + '/' + child_name.split('/')[-1]

def copy_asset_recursive(src, dst):
    """
    Copia recursivamente un asset de GEE (carpeta, colección, imagen o tabla).
    
    Args:
        src: Ruta del asset origen
        dst: Ruta del asset destino
    """
    info = ee.data.getAsset(src)
    asset_type = info["type"]
    
    print(f"\nProcesando: {src} ({asset_type})")

    # ---- CARPETAS ----
    if asset_type == "FOLDER":
        if not exists(dst):
            print(f"  → Creando carpeta: {dst}")
            ee.data.createFolder(dst)
        else:
            print(f"  → Carpeta ya existe: {dst}")

        children = ee.data.listAssets({"parent": src}).get("assets", [])
        print(f"  → Encontrados {len(children)} elementos")
        
        for child in children:
            child_src = child["name"]
            child_dst = join_path(dst, child_src)
            copy_asset_recursive(child_src, child_dst)

    # ---- IMAGE COLLECTION ----
    elif asset_type == "IMAGE_COLLECTION":
        if not exists(dst):
            print(f"  → Creando ImageCollection: {dst}")
            ee.data.createAsset({"type": "IMAGE_COLLECTION"}, dst)
        else:
            print(f"  → ImageCollection ya existe: {dst}")

        children = ee.data.listAssets({"parent": src}).get("assets", [])
        print(f"  → Copiando {len(children)} imágenes")
        
        for i, child in enumerate(children, 1):
            child_src = child["name"]
            child_dst = join_path(dst, child_src)
            print(f"     [{i}/{len(children)}] Copiando imagen: {child_src.split('/')[-1]}")
            try:
                ee.data.copyAsset(child_src, child_dst)
            except Exception as e:
                print(f"       Error: {e}")

    # ---- IMAGEN / TABLA / OTRO ----
    else:
        print(f"  → Copiando {asset_type}: {src}")
        try:
            ee.data.copyAsset(src, dst)
            print(f"     Éxito")
        except Exception as e:
            print(f"     Error: {e}")

print("Función de copia recursiva definida")

### 5.2 Usar la función de copia recursiva

In [ ]:
import ee
ee.Initialize()

# Definir origen y destino
src_folder = f'{USER_PATH}/carpeta_completa'
dst_folder = f'{PROJECT_PATH}/carpeta_completa_copia'

print(f"Iniciando copia recursiva:")
print(f"  Origen: {src_folder}")
print(f"  Destino: {dst_folder}")
print("="*60)

try:
    copy_asset_recursive(src_folder, dst_folder)
    print("\n" + "="*60)
    print("Copia recursiva completada exitosamente")
except Exception as e:
    print(f"\nError durante la copia: {e}")

## 6. Eliminar Assets - Operaciones Individuales

### 6.1 Eliminar una imagen o tabla individual

In [ ]:
import ee
ee.Initialize()

# Eliminar un asset individual (imagen o tabla)
asset_to_delete = f'{USER_PATH}/nombre_del_asset'

print(f"Eliminando: {asset_to_delete}")

try:
    ee.data.deleteAsset(asset_to_delete)
    print("Asset eliminado exitosamente")
except Exception as e:
    print(f"Error al eliminar: {e}")

### 6.2 Eliminar una ImageCollection vacía

In [ ]:
import ee
ee.Initialize()

# Eliminar una ImageCollection que está vacía
collection_id = f'{USER_PATH}/coleccion_vacia'

print(f"Eliminando colección vacía: {collection_id}")

try:
    ee.data.deleteAsset(collection_id)
    print("Colección eliminada exitosamente")
except Exception as e:
    print(f"Error: {e}")

### 6.3 Eliminar ImageCollection con todas sus imágenes



In [ ]:
import ee
ee.Initialize()

collection_id = f'{PROJECT_PATH}/coleccion_con_imagenes'

print(f"Eliminando ImageCollection: {collection_id}")

# 1. Obtener lista de imágenes
try:
    image_list = ee.ImageCollection(collection_id).toList(10000)
    count = image_list.size().getInfo()
    print(f"Total de imágenes: {count}\n")

    # 2. Borrar cada imagen
    for i in range(count):
        # Siempre obtener la imagen en índice 0 ya que la lista se reduce después de cada eliminación
        img = ee.Image(image_list.get(0))
        img_id = img.getInfo()['id']
        print(f"[{i+1}/{count}] Borrando imagen: {img_id}")
        try:
            ee.data.deleteAsset(img_id)
            print("  → Eliminada")
            # Actualizar la lista después de eliminar
            image_list = ee.ImageCollection(collection_id).toList(10000)
        except Exception as e:
            print(f"  → Error: {e}")

    # 3. Borrar la colección vacía
    print(f"\nBorrando colección vacía: {collection_id}")
    ee.data.deleteAsset(collection_id)
    print("Eliminación completa")
    
except Exception as e:
    print(f"Error: {e}")

### 6.4 Eliminar múltiples assets de una carpeta

In [ ]:
import ee
ee.Initialize()

# Eliminar todos los assets de una carpeta (sin eliminar la carpeta)
folder_path = f'{USER_PATH}/carpeta_a_limpiar'

print(f"Eliminando assets de: {folder_path}\n")

try:
    assets = ee.data.listAssets({'parent': folder_path})['assets']
    print(f"Total de assets a eliminar: {len(assets)}\n")
    
    for i, asset in enumerate(assets, 1):
        asset_id = asset['id']
        asset_type = asset['type']
        print(f"[{i}/{len(assets)}] Eliminando {asset_type}: {asset_id}")
        
        try:
            ee.data.deleteAsset(asset_id)
            print("  → Eliminado")
        except Exception as e:
            print(f"  → Error: {e}")
    
    print("\nProceso completado")
    
except Exception as e:
    print(f"Error: {e}")

## 7. Eliminar Estructuras Completas (Recursivo)

### 7.1 Función de eliminación recursiva básica

In [ ]:
import ee
ee.Initialize()

def delete_recursive(asset_path):
    """
    Elimina un asset de GEE de forma recursiva.
    
    Args:
        asset_path: Ruta del asset a eliminar (carpeta, colección, etc.)
    """
    
    # Listar contenidos del asset
    children = ee.data.listAssets({'parent': asset_path}).get('assets', [])
    
    print(f"\nProcesando: {asset_path}")
    print(f"  → {len(children)} elementos encontrados")

    for child in children:
        child_id = child["id"]
        asset_type = child["type"]

        print(f"\n  Elemento: {child_id} ({asset_type})")

        if asset_type == "FOLDER":
            # Carpeta: eliminar recursivamente
            delete_recursive(child_id)

        elif asset_type == "IMAGE_COLLECTION":
            print(f"    → Eliminando imágenes de la colección...")

            # Obtener lista de imágenes
            imgs = ee.data.listAssets({'parent': child_id}).get('assets', [])
            print(f"    → {len(imgs)} imágenes a eliminar")

            for i, img in enumerate(imgs, 1):
                img_id = img["id"]
                print(f"       [{i}/{len(imgs)}] Eliminando: {img_id.split('/')[-1]}")
                try:
                    ee.data.deleteAsset(img_id)
                except Exception as e:
                    print(f"         Error: {e}")

            print(f"    → Eliminando colección vacía: {child_id}")
            ee.data.deleteAsset(child_id)

        elif asset_type in ["IMAGE", "TABLE"]:
            print(f"    → Eliminando {asset_type.lower()}: {child_id}")
            try:
                ee.data.deleteAsset(child_id)
            except Exception as e:
                print(f"       Error: {e}")

        else:
            print(f"    → Tipo no manejado: {asset_type}")

    # Finalmente borrar el asset contenedor
    print(f"\n  → Eliminando contenedor: {asset_path}")
    try:
        ee.data.deleteAsset(asset_path)
        print(f"     Éxito")
    except Exception as e:
        print(f"     Error: {e}")

print("Función de eliminación recursiva definida")

### 7.2 Usar la función de eliminación recursiva

In [ ]:
import ee
ee.Initialize()

# ADVERTENCIA: Esta operación es irreversible
# Asegúrate de que quieres eliminar todo el contenido

target_folder = f'{USER_PATH}/carpeta_a_eliminar'

print("="*60)
print("ADVERTENCIA: Eliminación recursiva")
print("="*60)
print(f"Se eliminará: {target_folder}")
print("Esta operación es IRREVERSIBLE")
print("="*60)

# Descomenta la siguiente línea para ejecutar
# delete_recursive(target_folder)
# print("\nELIMINACIÓN RECURSIVA COMPLETADA")

print("\n(Código comentado por seguridad)")

### 7.3 Función de eliminación recursiva con barra de progreso

Versión mejorada con feedback visual usando tqdm.

In [ ]:
import ee
ee.Initialize()

# Instalar tqdm si no está disponible
try:
    from tqdm import tqdm
except ImportError:
    print("Instalando tqdm...")
    import sys
    !{sys.executable} -m pip install tqdm
    from tqdm import tqdm

def delete_recursive_progress(asset_path, level=0):
    """
    Elimina recursivamente assets de GEE con barra de progreso.
    
    Args:
        asset_path: Ruta del asset a eliminar
        level: Nivel de profundidad (para indentación)
    """
    
    # Obtener assets dentro de la carpeta/colección
    children = ee.data.listAssets({'parent': asset_path}).get('assets', [])

    indent = "  " * level
    print(f"{indent}Explorando: {asset_path} ({len(children)} assets)")

    # Barra de progreso para los contenidos
    for child in tqdm(children, desc=f"{indent}Nivel {level}", leave=True):
        child_id = child["id"]
        asset_type = child["type"]

        print(f"{indent}→ {child_id.split('/')[-1]} ({asset_type})")

        if asset_type == "FOLDER":
            # Carpeta: recursión
            delete_recursive_progress(child_id, level + 1)

        elif asset_type == "IMAGE_COLLECTION":
            print(f"{indent}   → Eliminando imágenes...")

            imgs = ee.data.listAssets({'parent': child_id}).get('assets', [])

            # Barra de progreso para imágenes
            for img in tqdm(imgs, desc=f"{indent}   Imágenes", leave=False):
                img_id = img["id"]
                try:
                    ee.data.deleteAsset(img_id)
                except Exception as e:
                    print(f"{indent}      Error: {e}")

            print(f"{indent}   → Eliminando colección vacía")
            ee.data.deleteAsset(child_id)

        elif asset_type in ["IMAGE", "TABLE"]:
            try:
                ee.data.deleteAsset(child_id)
            except Exception as e:
                print(f"{indent}   Error: {e}")

        else:
            print(f"{indent}   Tipo no manejado: {asset_type}")

    # Borrar el contenedor
    print(f"{indent}→ Eliminando: {asset_path}")
    try:
        ee.data.deleteAsset(asset_path)
    except Exception as e:
        print(f"{indent}  Error: {e}")

print("Función de eliminación con progreso definida")

### 7.4 Usar la función de eliminación con progreso

In [ ]:
import ee
ee.Initialize()

# ADVERTENCIA: Esta operación es irreversible
target_folder = f'{USER_PATH}/carpeta_a_eliminar'

print("="*60)
print("ELIMINACIÓN RECURSIVA CON PROGRESO")
print("="*60)
print(f"Target: {target_folder}")
print("="*60)

# Descomenta para ejecutar
# delete_recursive_progress(target_folder)
# print("\n✓ PROCESO COMPLETADO")

print("\n(Código comentado por seguridad)")

## 8. Utilidades Adicionales

### 8.1 Crear carpeta

In [ ]:
import ee
ee.Initialize()

# Crear una carpeta nueva
new_folder = f'{PROJECT_PATH}/nueva_carpeta'

print(f"Creando carpeta: {new_folder}")

try:
    ee.data.createFolder(new_folder)
    print("Carpeta creada exitosamente")
except Exception as e:
    print(f"Error (puede que ya exista): {e}")

### 8.2 Crear ImageCollection vacía

In [ ]:
import ee
ee.Initialize()

# Crear una ImageCollection vacía
new_collection = f'{PROJECT_PATH}/nueva_coleccion'

print(f"Creando ImageCollection: {new_collection}")

try:
    ee.data.createAsset({'type': 'ImageCollection'}, new_collection)
    print("ImageCollection creada exitosamente")
except Exception as e:
    print(f"Error (puede que ya exista): {e}")

### 8.3 Verificar si un asset existe

In [ ]:
import ee
ee.Initialize()

def asset_exists(asset_id):
    """Verifica si un asset existe en GEE."""
    try:
        ee.data.getAsset(asset_id)
        return True
    except:
        return False

# Ejemplo de uso
asset_to_check = f'{USER_PATH}/nombre_asset'

if asset_exists(asset_to_check):
    print(f"El asset existe: {asset_to_check}")
else:
    print(f"El asset NO existe: {asset_to_check}")

### 8.4 Contar elementos en una carpeta o colección

In [ ]:
import ee
ee.Initialize()

def count_assets(parent_path):
    """Cuenta los assets en una carpeta o colección."""
    try:
        assets = ee.data.listAssets({'parent': parent_path})['assets']
        return len(assets)
    except Exception as e:
        print(f"Error: {e}")
        return 0

# Ejemplo de uso
path_to_count = f'{USER_PATH}/carpeta'
count = count_assets(path_to_count)
print(f"Total de assets en {path_to_count}: {count}")

### 8.5 Listar todos los tipos de assets en una ubicación

In [ ]:
import ee
ee.Initialize()

def list_assets_by_type(parent_path):
    """Lista y agrupa assets por tipo."""
    try:
        assets = ee.data.listAssets({'parent': parent_path})['assets']
        
        by_type = {}
        for asset in assets:
            asset_type = asset['type']
            if asset_type not in by_type:
                by_type[asset_type] = []
            by_type[asset_type].append(asset['id'])
        
        print(f"Assets en {parent_path}:\n")
        for asset_type, items in by_type.items():
            print(f"{asset_type}: {len(items)}")
            for item in items:
                print(f"  - {item.split('/')[-1]}")
            print()
        
        return by_type
    
    except Exception as e:
        print(f"Error: {e}")
        return {}

# Ejemplo de uso
list_assets_by_type(USER_PATH)

## 9. Ejemplos de Casos de Uso Completos

### 9.1 Migrar todo de usuario a proyecto

In [ ]:
import ee
ee.Initialize()

# Migrar una carpeta completa de usuario a proyecto
src = f'{USER_PATH}/datos_a_migrar'
dst = f'{PROJECT_PATH}/datos_migrados'

print("Iniciando migración completa")
print(f"Origen: {src}")
print(f"Destino: {dst}")
print("="*60)

try:
    # Copiar todo recursivamente
    copy_asset_recursive(src, dst)
    print("\n" + "="*60)
    print("Migración completada exitosamente")
    print("\nNOTA: Los datos originales NO han sido eliminados.")
    print("Si deseas eliminarlos, usa delete_recursive()")
    
except Exception as e:
    print(f"\nError durante la migración: {e}")

### 9.2 Backup de assets antes de eliminar

In [ ]:
import ee
ee.Initialize()

# Crear backup antes de eliminar
original = f'{USER_PATH}/datos_importantes'
backup = f'{USER_PATH}/datos_importantes_backup'

print("Creando backup...")
print(f"Original: {original}")
print(f"Backup: {backup}")
print("="*60)

try:
    # Copiar a backup
    copy_asset_recursive(original, backup)
    print("\nBackup creado exitosamente")
    print(f"\nAhora puedes eliminar {original} de forma segura")
    print(f"El backup está en: {backup}")
    
except Exception as e:
    print(f"\nError al crear backup: {e}")

### 9.3 Reorganizar estructura de carpetas

In [ ]:
import ee
ee.Initialize()

# Reorganizar copiando a nueva estructura y luego eliminando la antigua
old_structure = f'{USER_PATH}/estructura_antigua'
new_structure = f'{PROJECT_PATH}/estructura_nueva'

print("Reorganizando estructura...")
print(f"De: {old_structure}")
print(f"A: {new_structure}")
print("="*60)

try:
    # 1. Copiar a nueva ubicación
    print("\n1. Copiando a nueva estructura...")
    copy_asset_recursive(old_structure, new_structure)
    
    print("\n2. Copia completada")
    print("\nNOTA: La estructura antigua NO ha sido eliminada.")
    print("Verifica que todo esté correcto en la nueva ubicación")
    print("antes de eliminar la antigua.")
    print(f"\nPara eliminar: delete_recursive('{old_structure}')")
    
except Exception as e:
    print(f"\nError: {e}")

---

## Notas Importantes

### Limitaciones y Consideraciones

1. **Límites de API**: GEE tiene límites en el número de operaciones por minuto. Si trabajas con muchos assets, considera agregar pausas entre operaciones.

2. **Permisos**: Asegúrate de tener los permisos necesarios tanto en el origen como en el destino.

3. **Operaciones irreversibles**: Las eliminaciones son permanentes. Siempre haz backups antes de eliminar datos importantes.

4. **ImageCollections grandes**: Para colecciones con más de 10,000 imágenes, considera usar paginación.

5. **Rutas relativas**: Las rutas en GEE siempre deben ser absolutas (incluir el prefijo completo).

### Buenas Prácticas

1. **Prueba primero**: Antes de operaciones masivas, prueba con un subconjunto pequeño.

2. **Backups**: Siempre haz backup antes de eliminar o mover datos importantes.

3. **Verifica destinos**: Asegúrate de que las carpetas/colecciones destino existen antes de copiar.

4. **Monitorea cuotas**: Revisa tus cuotas de almacenamiento en GEE regularmente.

5. **Usa variables**: Define rutas comunes al inicio del notebook para evitar errores de tipeo.

### Soporte

Para más información sobre la API de Google Earth Engine:
- Documentación oficial: https://developers.google.com/earth-engine
- Guía de gestión de assets: https://developers.google.com/earth-engine/guides/asset_manager